In [126]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [127]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [128]:
data = pd.read_csv("botswana.csv", sep="\t") 
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [129]:
data.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [130]:
data.dropna().shape

(1834, 15)

Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.

Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.

Замените NaN в признаке agefm на cagefm=0.

У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).


Сколько осталось пропущенных значений в признаке heduc?

In [131]:
data['nevermarr'] = data['agefm'].isnull().astype(int)
data.drop('evermarr', axis=1, inplace=True)
data.agefm.fillna(0, inplace=True)
# data[data['nevermarr'] == 1]['heduc'].fillna(-1, inplace=True)

In [132]:
type(data[data['nevermarr'] == 1]['heduc'].isnull())

pandas.core.series.Series

In [133]:
for ind, flag in data[data['nevermarr'] == 1]['heduc'].isnull().iteritems():
    if flag:
        data.heduc[ind] = -1

/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [134]:
sum(data.heduc.isnull())

123

# 8
Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик).

In [135]:
data['idlnchld_noans'] = data['idlnchld'].isnull().astype(int)
data['idlnchld'].fillna(-1, inplace=True)

In [136]:
data['heduc_noans'] = data['heduc'].isnull().astype(int)
data['heduc'].fillna(-2, inplace=True)

In [137]:
data['usemeth_noans'] = data['usemeth'].isnull().astype(int)
data['usemeth'].fillna(-2, inplace=True)

In [138]:
data.dropna(inplace=True)

In [139]:
data.shape

(4348, 18)

In [140]:
data.shape[0]*data.shape[1]

78264

In [141]:
data.columns.values

array(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'], dtype=object)

In [142]:
m1 = smf.ols('ceb ~ idlnchld + age + educ + religion + knowmeth + usemeth +\
                agefm + heduc + urban + electric + radio + tv + bicycle +\
                nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Wed, 22 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:40:25   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [143]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [144]:
m2 = smf.ols('ceb ~ idlnchld + age + educ + religion + knowmeth + usemeth +\
                agefm + heduc + urban + electric + radio + tv + bicycle +\
                nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Wed, 22 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:40:44   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [145]:
data2 = data.drop(['religion', 'radio', 'tv'], axis=1)

In [146]:
m3 = smf.ols('ceb ~ idlnchld + age + educ + knowmeth + usemeth +\
                agefm + heduc + urban + electric + bicycle +\
                nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data2)
fitted = m3.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Wed, 22 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:40:48   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [147]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [148]:
m4 = smf.ols('ceb ~ idlnchld + age + educ + knowmeth + usemeth +\
                agefm + heduc + urban + electric + bicycle +\
                nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data2)
fitted = m4.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Wed, 22 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:40:53   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [149]:
print "F=%f, p=%f, k1=%f" % m2.fit(cov_type='HC1').compare_f_test(m4.fit(cov_type='HC1'))

F=0.919236, p=0.467231, k1=5.000000


/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1540: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


In [150]:
data3 = data.drop(['usemeth_noans', 'usemeth'], axis=1)

In [154]:
m5 = smf.ols('ceb ~ idlnchld + age + educ + knowmeth +\
                agefm + heduc + urban + electric + bicycle +\
                nevermarr + idlnchld_noans + heduc_noans + religion + radio + tv', 
             data=data3)
fitted = m5.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Wed, 22 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:41:45   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [155]:
print "F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m5.fit())

F=27.194290, p=0.000000, k1=7.000000


In [156]:
m2.fit(cov_type='HC1').compare_f_test(m5.fit())

/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1540: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(27.194289901620241, 8.6587076625307275e-37, 7.0)